IMPORTANT:this TF file Needs MP_Data folder from data collection step

In [ ]:
#To remove leftovers from previous training
#!rm -rf MP_Data_Child
#!rm -rf MP_Data_Child.zip
#!rm -rf action.h5
#!rm -rf Logs

# Preprocess

In [ ]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
#Remove annoying DS_store
#%cd MP_Data
#!rm -rf .DS_Store
#%cd ..

In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
#import mediapipe as mp

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [6]:
#Upload the zipped data
#!unzip file_location
!unzip MP_AlphaC7.zip

In [ ]:
#%cd MP_Data

In [7]:
# Path for exported data, numpy arrays
Folder='MP_AlphaC7'

DATA_PATH = os.path.join(Folder) 

#Sorting to negate shuffling of dirs
dirs=os.listdir(Folder)
dirs.sort()

# Actions that we try to detect
actions = np.array(dirs)

# Thirty videos worth of data
no_sequences = len(os.listdir(os.path.join(Folder,actions[0])))

# Videos are going to be 24 frames in length
sequence_length = len(os.listdir(os.path.join(Folder,actions[0],str(0))))


In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
label_map

{'A': 0, 'B': 1, 'C': 2}

In [10]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
#sequences

In [11]:
X = np.array(sequences)
X.shape

(303, 1, 960)

In [12]:
y = to_categorical(labels).astype(int)
y.shape

(303, 3)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [14]:
X_train.shape, X_test.shape

((287, 1, 960), (16, 1, 960))

# Train

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

TensorBoard

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

LSTM Training Model

In [17]:
X.shape

(303, 1, 960)

In [18]:
model = Sequential()
#3 sets of LSTM layers
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(X.shape[1],X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
#2 Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#Softmax function
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
#(X.shape[1],X.shape[2])

In [ ]:
#actions.shape[0]

In [ ]:
#sidework
#res = [.7, .3]
#actions[np.argmax(res)]

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
9/9 [==============================] - 4s 10ms/step - loss: 1.0989 - categorical_accuracy: 0.3275
Epoch 2/100
9/9 [==============================] - 0s 10ms/step - loss: 1.0976 - categorical_accuracy: 0.4286
Epoch 3/100
9/9 [==============================] - 0s 10ms/step - loss: 1.0910 - categorical_accuracy: 0.5436
Epoch 4/100
9/9 [==============================] - 0s 10ms/step - loss: 1.0543 - categorical_accuracy: 0.6167
Epoch 5/100
9/9 [==============================] - 0s 9ms/step - loss: 0.9038 - categorical_accuracy: 0.5819
Epoch 6/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6530 - categorical_accuracy: 0.6376
Epoch 7/100
9/9 [==============================] - 0s 10ms/step - loss: 0.5317 - categorical_accuracy: 0.6551
Epoch 8/100
9/9 [==============================] - 0s 9ms/step - loss: 0.5054 - categorical_accuracy: 0.6481
Epoch 9/100
9/9 [==============================] - 0s 10ms/step - loss: 0.5157 - categorical_accuracy: 0.6481
Epoch 10/100


KeyboardInterrupt: ignored

In [ ]:
#a little bit more training
#model.fit(X_train, y_train, epochs=100)

In [ ]:
#model.fit(X_train, y_train, epochs=100)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             262400    
                                                                 
 lstm_1 (LSTM)               (None, 1, 128)            98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 416,963
Trainable params: 416,963
Non-trai

In [ ]:
#tensorboard log dir
#!zip -r /content/file.zip /content/Logs
#from google.colab import files
#files.download("/content/file.zip")
#go to the correct command directory for Logs and type in the following in the command line..
#cd ..filedir
#tensorboard --logdir=.
#..then copy pase the URL to a browser

# Testing

In [22]:
result=model.predict(X_test)

1/1 [==============================] - 0s 447ms/step


In [23]:
actions[np.argmax(result[0])]

'C'

In [24]:
#Self evaluation
#i=0
#(actions[np.argmax(result[i])]==
#actions[np.argmax(y_test[i])])

In [25]:
model.save('modelAlpha.h5')

In [ ]:
#to save to local device
#from google.colab import files
#files.download("/content/action.h5")

In [ ]:
#To delete model
#del model

In [ ]:
#model.load_weights('action.h5')

# Evaluation

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 23ms/step


In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[ 8,  0],
        [ 0,  8]],

       [[13,  0],
        [ 0,  3]],

       [[11,  0],
        [ 0,  5]]])

In [ ]:
accuracy_score(ytrue, yhat)

1.0